In [1]:
import gensim
assert gensim.models.word2vec.FAST_VERSION > -1

In [2]:
import codecs

stop='../stopwords.txt'
emojiSynonyms='../emojiWords.txt'
stopList=[]
emojiMap={}
emojis=[]
f=codecs.open(stop,'r', encoding='utf-8')

for line in f.readlines():
    stopList.append(line.strip())

f=codecs.open(emojiSynonyms,'r', encoding='utf-8')
for line in f.readlines():
    emoji, meaning= line.strip().split()
    emojiMap[emoji.strip()]=' '.join(meaning.strip().split(','))
    emojis.append(emoji.strip())
        
def NormalizeDates(title):    
    fullDecades=['1930','1940','1950','1960','1970','1980','1990','2000','2010']
    truncDecades=['30','40','50','60','70','80','90','00','10']
    year=['2001','2002','2003','2004','2005','2006','2007','2008','2009','2011','2012','2013','2014','2015','2016','2017']
    truncYear=['02','03','04','05','06','07','08','09','11','12','13','14','15','16','17']
    seasons=['spring', 'summer','fall','winter','sommer', 'autumn', 'verano' ]
    months=['january','jan','february','feb','march','april','may','june','july','august','aug','spetember','sep','october','oct','novemeber','nov','december','dec']
    monthPairs=[('january','jan'),('february','feb'),('august','aug'),('september','sept'), ('september','sep'),('october','oct'),('november','nov'),('december','dec')]

    title=title.lower().strip()
    
    for decade in fullDecades:
        if decade in title:
            newTitle=''
            for word in title.split():
                if decade in word:
                    word=word.replace(decade+"'",decade)
                    word=word.replace(decade+"s",decade)
                    word=word.replace(decade,' '+decade+'s ')
                newTitle=newTitle+' '+word
            #print title+'------'+newTitle
            title=newTitle.strip()

    for truncDecade in truncDecades:
        if truncDecade in title and ('19'+truncDecade not in title) and ('20'+truncDecade not in title):
            newTitle=''
            for word in title.split():
                if truncDecade+"'" in word  or truncDecade+"s" in word or truncDecade+u"´s" in word or truncDecade+"'s" in word or truncDecade+" s " in word or truncDecade+"ies" in word:
                    word=word.replace(truncDecade+"'",truncDecade)
                    word=word.replace(truncDecade+"s",truncDecade)
                    word=word.replace(truncDecade+" s",truncDecade)
                    word=word.replace(truncDecade+u"´s",truncDecade)
                    word=word.replace(truncDecade+"ies",truncDecade)
                    if truncDecade in truncDecades[:7]:
                        century='19'
                    else:
                        century='20'
                    word=word.replace(truncDecade,' '+century+truncDecade+'s ')
                newTitle=newTitle+' '+word
            title=newTitle.strip()
    if title.strip() in ['70-80','70,80,90','90/00', '70 80 90', '90 00', '80 90', '80-90','90-00', '60-70', '70 80']:
        for truncDecade in truncDecades:
            if truncDecade in truncDecades[:7]:
                century='19'
            else:
                century='20'
            title=title.replace(truncDecade,' '+century+truncDecade+'s ' )
    title=title.strip()
    for yr in year:
        if yr in title:
            title=title.replace(yr,' '+yr+' ')
            
    for yr in truncYear:
        if yr in title and '20'+yr not in title:
            if "'"+yr in title:
                #print title
                title=title.replace("'"+yr,' 20'+yr+' ')
                #print title
            if yr+"'" in title:
                #print title
                title=title.replace(yr+"'",' 20'+yr+' ')
                #print title
            if '2k'+yr in title or '2K'+yr in title:
                #print title
                title=title.replace('2k'+yr,' 20'+yr+' ')
                #print title
            if '-'+yr in title :
                #print title
                title=title.replace('-'+yr,' 20'+yr+' ')
                #print title
        if yr in title and '20'+yr not in title:
            for season in seasons:
                if season in title:
                    #print title
                    title=title.replace(yr,' 20'+yr+' ')
                    #print title
            for month in months:
                if month in title and '20'+yr not in title:
                    #print title
                    title=title.replace(yr,' 20'+yr+' ')
                    #print '-'+title
    for month,shortMonth in monthPairs:
        newTitle=''
        if shortMonth in title and month not in title:
            #print title
            for word in title.split():
                if shortMonth== word:
                    newTitle=newTitle+' '+month
                else:
                    newTitle=newTitle+' '+word
            title=newTitle
            #print '-'+title
    title=title.strip()
    title=' '.join(title.split())
    return title

def handleEmojis(title):
    for emo in emojis:
        if emo in title:
            title=title.replace(emo,' ')
            title=title+' '+emojiMap[emo]+' '
            
    title=title.replace(u'\U0001f3fc','')
    title=title.replace(u'\U0001f3fd','')
    title=title.replace(u'\U0001f3fb','')
    title=title.replace(u'\U0001f3fe','')
    title=title.replace(u'\u200d','')
    title=title.replace(u'\ufe0f','')
    title=title.replace(u'oshrug','shrug')
    title=title.replace(u'\ufffd','')
    title=title.replace(u'\U0001f37b','')
    title=title.replace(u'\u200d','')
    title=title.replace(u'\u2640','')
    title=title.replace(u'\u2642','')
    title=title.replace(u'\U0001f3b6','')
    title=title.replace(u'\u2728','')
    title=title.replace(u'\U0001f449','')
    title=title.replace(u'\U0001f3ff','')
    title=title.replace(u'\U0001f38a','')
    title=title.replace(u'\U0001f445','')
    title=title.replace(u'\U0001f608','')
    title=title.replace(u'\U0001f381','')
    title=title.replace(u'\U0001f60f','')
    title=title.replace(u'\U0001f4a8','')
    title=title.replace(u'�','')
    title=title.replace('<3',' heart love ')
    title=title.replace(':)',' smile happy ')
    title=title.replace(';)',' smirk happy ')
    title=title.replace(':-)',' smile happy ')
    title=title.replace(': )',' smile happy ')
    title=title.replace(u'😋',' smile happy ')
    title=title.replace(u'\u263a\ufe0f',' smile happy ')
    title=title.replace('r&b',' randb ')
    title=title.replace('r & b',' randb ')
    title=title.replace('rnb',' randb ')
    title=title.replace(u'•',' ')
    title=title.replace(u'\u263a\ufe0f',' death poison ')
    title=title.replace(u'\u2615\ufe0f',' coffee tea morning ')
    title=title.replace(u'💩',' poop ')
    title=title.strip()
    title=' '.join(title.split())
    return title
# # ' & - $ . : ! / () * ,
import re
def normalize_nameTitle(name):
    name = name.lower()
    name = NormalizeDates(name)
    name = handleEmojis(name)
    name = name.replace('happysmile','happy smile')
    name = re.sub(r"[.,\/#\'?\&\-!$%\^\*;:{}=\_`~()@]", ' ', name)
    name = re.sub(r'\s+', ' ', name).strip()
    return name
def normalize_name(name):
    name = name.lower()
    name = re.sub(r"[.,\/#\'?\&\-!$%\^\*;:{}=\_`~()@]", ' ', name)
    name = re.sub(r'\s+', ' ', name).strip()
    return name

import nltk
def titlePlusBigrams(title):
    bigrm = list(nltk.bigrams(title.split()))
    bis=''
    for x1,x2 in bigrm:
        bis=bis+' '+x1+x2
    return title+' '+bis.strip()

In [3]:
import sys
import json
import re
import collections
import os
import itertools as it
import gensim
import pickle

path='../../data'
backgroundFiles=pickle.load(open('./SplitsInformation/filenamesBackGroundData.pkl','rb'))

class Playlists(object):
    def __init__(self, dirname):
        self.dirname = dirname
 
    def __iter__(self):
        count=0
        wlen=20
        
        filenames = os.listdir(self.dirname)
        for filename in sorted(filenames):
            if filename.startswith("mpd.slice.") and filename.endswith(".json"):
                fullpath = os.sep.join((path, filename))
                f = open(fullpath)
                js = f.read()
                f.close()
                mpd_slice = json.loads(js)
                for playlist in mpd_slice['playlists']:
                    name=normalize_nameTitle(playlist['name'])
                    name1 =titlePlusBigrams(name)
                    trackSet=[]
                    for track in playlist['tracks']:
                        trackSet.append(track['track_uri'].replace('spotify:track:',''))
                    trackSet1=trackSet[:]
                    cnt=0
                    l=len(trackSet)
                    if l<wlen:
                        trackSet.insert(l/2,name)
                        trackSet1.insert(l/2,name1)
                    for i in range(0,l,):
                        trackSet.insert(i+cnt, name)
                        trackSet1.insert(i+cnt, name1)
                        cnt=cnt+1
                    #for pair in list(it.combinations(trackSet, 2)):
                    yield list(trackSet)
                    yield list(trackSet1)
                count += 1
                if count%50 ==0:
                    print "processing "+ str(count) +'.....'

sentences = Playlists(path)

model = gensim.models.Word2Vec(sentences,sg=0, window=20, min_count=1, size=200, workers=96, iter=20)
mname='./W2VModels/w2v_TitleAndTracks_All1MBG_CBOW_w20_200em.bin'
model.save(mname)
'''
for wlen in [2,5,10,20,30]:                    
    model = gensim.models.Word2Vec(sentences,sg=0, window=wlen, min_count=1, size=200, workers=16, iter=5)
    mname='./w2vModels/w2v_CBOW_w'+str(wlen)+'_200em.bin'
    model.save(mname)
    model = gensim.models.Word2Vec(sentences,sg=1, window=wlen, min_count=1, size=200, workers=16, iter=5)
    mname='./w2vModels/w2v_SG_w'+str(wlen)+'_200em.bin'
    model.save(mname)
'''

processing 50.....
processing 100.....
processing 150.....
processing 200.....
processing 250.....
processing 300.....
processing 350.....
processing 400.....
processing 450.....
processing 500.....
processing 550.....
processing 600.....
processing 650.....
processing 700.....
processing 750.....
processing 800.....
processing 850.....
processing 900.....
processing 950.....
processing 1000.....
processing 50.....
processing 100.....
processing 150.....
processing 200.....
processing 250.....
processing 300.....
processing 350.....
processing 400.....
processing 450.....
processing 500.....
processing 550.....
processing 600.....
processing 650.....
processing 700.....
processing 750.....
processing 800.....
processing 850.....
processing 900.....
processing 950.....
processing 1000.....
processing 50.....
processing 100.....
processing 150.....
processing 200.....
processing 250.....
processing 300.....
processing 350.....
processing 400.....
processing 450.....
processing 500.....
p

processing 550.....
processing 600.....
processing 650.....
processing 700.....
processing 750.....
processing 800.....
processing 850.....
processing 900.....
processing 950.....
processing 1000.....


"\nfor wlen in [2,5,10,20,30]:                    \n    model = gensim.models.Word2Vec(sentences,sg=0, window=wlen, min_count=1, size=200, workers=16, iter=5)\n    mname='./w2vModels/w2v_CBOW_w'+str(wlen)+'_200em.bin'\n    model.save(mname)\n    model = gensim.models.Word2Vec(sentences,sg=1, window=wlen, min_count=1, size=200, workers=16, iter=5)\n    mname='./w2vModels/w2v_SG_w'+str(wlen)+'_200em.bin'\n    model.save(mname)\n"